In [1]:
# First implementation of multi-objective optimization of infill patterns
# using pymoo optimization library with structural objectives based on
# FEA evaluation in FEniCS framework with Gmsh meshing

import numpy as np
import matplotlib.pyplot as plt
import pathlib
from pathlib import Path

from pymoo.core.problem import ElementwiseProblem
from pymoo.algorithms.moo.nsga2 import NSGA2

from pymoo.operators.sampling.rnd import BinaryRandomSampling
from pymoo.operators.crossover.hux import HalfUniformCrossover
from pymoo.operators.crossover.ux import UniformCrossover
from pymoo.operators.mutation.bitflip import BitflipMutation

from pymoo.termination import get_termination
from pymoo.optimize import minimize

import meshio
import gmsh
from dolfin import *

def create_mesh(mesh, cell_type):
    cells = mesh.get_cells_type(cell_type)
    cell_data = mesh.get_cell_data("gmsh:geometrical", cell_type)
    out_mesh = meshio.Mesh(points=mesh.points[:,:2], cells={cell_type: cells}, cell_data={"name_to_read": [cell_data]})
    return out_mesh


def extract_mesh_data(mesh_to_load):
    msh = meshio.read(mesh_to_load)

    # filepath management
    cwd = Path.cwd()
    temp_dir = cwd.joinpath("temp_mesh")   # temp dir for loading meshes
    temp_dir.mkdir(exist_ok=True)          # create parent dir if it does not exist


    infill_triangles = create_mesh(msh, "triangle")
    infill_lines = create_mesh(msh, "line")
    meshio.write(temp_dir / "mesh.xdmf", infill_triangles)
    meshio.write(temp_dir / "mf.xdmf", infill_lines)

    infill_mesh = Mesh()
    mvc = MeshValueCollection("size_t", infill_mesh, infill_mesh.topology().dim())
    with XDMFFile(str(temp_dir / "mesh.xdmf")) as infile:
        infile.read(infill_mesh)
        infile.read(mvc, "name_to_read")
    cf = cpp.mesh.MeshFunctionSizet(infill_mesh, mvc)

    mvc = MeshValueCollection("size_t", infill_mesh, infill_mesh.topology().dim() - 1)
    with XDMFFile(str(temp_dir / "mf.xdmf")) as infile:
        infile.read(mvc, "name_to_read")
    mf = cpp.mesh.MeshFunctionSizet(infill_mesh, mvc)

    infill_connect = infill_mesh.cells()       # triangle connectivity of the infill mesh
    infill_coords = infill_mesh.coordinates()  # coordinates within the infill mesh
    infill_coords = np.round(infill_coords, 4) # rounding the coordinates
    num_el = len(infill_connect)               # number of infill elements

    return infill_connect, infill_coords, num_el

def extract_mesh_data_fenics(mesh_to_load):
    msh = meshio.read(mesh_to_load)

    # filepath management
    cwd = Path.cwd()
    temp_dir = cwd.joinpath("temp_mesh")   # temp dir for loading meshes
    temp_dir.mkdir(exist_ok=True)          # create parent dir if it does not exist

    triangle_mesh = create_mesh(msh, "triangle")
    line_mesh = create_mesh(msh, "line")
    meshio.write(temp_dir / "mesh.xdmf", triangle_mesh)
    meshio.write(temp_dir / "mf.xdmf", line_mesh)

    mesh = Mesh()
    mvc = MeshValueCollection("size_t", mesh, mesh.topology().dim())
    with XDMFFile(str(temp_dir / "mesh.xdmf")) as infile:
        infile.read(mesh)
        infile.read(mvc, "name_to_read")
    cf = cpp.mesh.MeshFunctionSizet(mesh, mvc)

    mvc = MeshValueCollection("size_t", mesh, mesh.topology().dim() - 1)
    with XDMFFile(str(temp_dir / "mf.xdmf")) as infile:
        infile.read(mvc, "name_to_read")
    mf = cpp.mesh.MeshFunctionSizet(mesh, mvc)

    return mesh, mvc, cf, mf

def generate_infill_file(meshpath, infill_vec, infill_connect, infill_coords, verbosity=1, run_GUI=False):
    size = 8  # physical dimension of the side
    P = 1      # number of perimeters
    width, height = size, size       # setting up the square
    num_el_x, num_el_y = size, size  # adjust the num of elements
    dx, dy = width / num_el_x, height / num_el_y

    # initialize Gmsh
    if not gmsh.is_initialized():
        gmsh.initialize()
        gmsh.option.setNumber('General.Verbosity', verbosity)

    gmsh.model.add(meshpath.stem)  # "smallSquare8-vecEl-substractive"

    # preparing parameters
    lc = 2 * dx     # mesh size

    # defining a namespace
    gmo = gmsh.model.occ  # gmo ~ gmsh.model.opencascade
    gmg = gmsh.model.geo  # gmg ~ gmsh.model.geo

    # defining the points
    gmo.addPoint(0, 0, 0, lc, 1)
    gmo.addPoint(0, size, 0, lc, 2)
    gmo.addPoint(size, size, 0, lc, 3)
    gmo.addPoint(size, 0, 0, lc, 4)

    # adding lines
    gmo.addLine(1, 2, 1)
    gmo.addLine(2, 3, 2)
    gmo.addLine(3, 4, 3)
    gmo.addLine(4, 1, 4)

    # defining a new surface:
    gmo.addCurveLoop([1, 2, 3, 4], 20)
    surf = gmo.addPlaneSurface([20])  # full surface
    gmo.synchronize()

    # meshing the transfinite surface
    gmsh.model.mesh.setTransfiniteCurve(1, num_el_x + 1)
    gmsh.model.mesh.setTransfiniteCurve(2, num_el_x + 1)
    gmsh.model.mesh.setTransfiniteCurve(3, num_el_x + 1)
    gmsh.model.mesh.setTransfiniteCurve(4, num_el_x + 1)
    gmo.synchronize()
    gmsh.model.mesh.setTransfiniteSurface(surf, arrangement="Alternate", cornerTags=[1,2,3,4])


    # subtracting individual infill elements
    infill_points = []   # list of point tags inside the infill
    for x, y in infill_coords:
        infill_points.append(gmo.addPoint(x, y, 0, lc))  # add each point

    infill_lines = []   # list of line tags inside the infill
    infill_curves = []  # list of curve tags inside the infill
    infill_surfs = []   # list of surface tags
    for i, tri in enumerate(infill_connect):   # get each point in each triangle
        if infill_vec[i] == 0:                 # only if is to be deleted (0: not included)
            p1, p2, p3 = tri[0], tri[1], tri[2]
            tp1 = infill_points[p1]  # tag of p1
            tp2 = infill_points[p2]  # tag of p2
            tp3 = infill_points[p3]  # tag of p3
            l1 = gmo.addLine(tp1, tp2)  # tag of line 1
            l2 = gmo.addLine(tp2, tp3)  # tag of line 2
            l3 = gmo.addLine(tp3, tp1)  # tag of line 3
            xp1, yp1 = infill_coords[p1, 0], infill_coords[p1, 1]  # getting coords of p1
            xp2, yp2 = infill_coords[p2, 0], infill_coords[p2, 1]  # getting coords of p2
            xp3, yp3 = infill_coords[p3, 0], infill_coords[p3, 1]  # getting coords of p3
            infill_lines.append(l1)
            infill_lines.append(l2)
            infill_lines.append(l3)
            c1 = gmo.addCurveLoop([l1, l2, l3])
            s1 = gmo.addPlaneSurface([c1])  # create the surface
            infill_curves.append(c1)
            infill_surfs.append(s1)
    gmo.synchronize()

    surfs_to_cut = [(2, tag) for tag in infill_surfs]  # list of all surfs
    if len(surfs_to_cut) > 0:
        # substract all in one
        outlab = gmo.cut([(2, surf)], surfs_to_cut, removeObject=True, removeTool=True)
        surf = outlab[0][0][1]
        gmo.synchronize()

    # extract the curve boundary
    Boundary = [tg for tp, tg in gmsh.model.getBoundary([(2, surf)])]

    # adding physical groups
    gmsh.model.addPhysicalGroup(1, Boundary, name="SurfaceLoop") # curves physical group
    gmsh.model.addPhysicalGroup(2, [surf], name="PerimeterSurface")   # surfaces physical group
    gmo.synchronize()

    # global meshing options
    gmsh.option.setNumber("Mesh.Algorithm", 6)  # Frontal-Delaunay for 2D meshes


    # generate the mesh
    gmo.synchronize()
    gmsh.model.mesh.generate(2)

    # saving the mesh
    gmsh.write(str(meshpath))

    # Launch the GUI
    if run_GUI:
        gmo.synchronize()
        gmsh.fltk.run()

    # finish the GMSH session
    gmsh.finalize()



In [2]:
# FEniCS functions setup

# boundary connection setup
tol = 2
def clamped_middle(x, on_boundary): # beam is only fixed in the middle
    #return on_boundary and near(x[0], 0, tol) and near(x[1], 20, tol)#x[0] < tol
    return on_boundary and near(x[0], 0 + 0.5, tol) and near(x[1], 5, tol)#x[0] < tol

# Define strain
def epsilon(u):
    return 0.5*(nabla_grad(u) + nabla_grad(u).T)
    # return sym(nabla_grad(u))

# Define stress
def sigma(u, lambda_, d, mu):
    return lambda_*nabla_div(u)*Identity(d) + 2*mu*epsilon(u)

# Delta function for point load
class Delta(UserExpression):
    def __init__(self, eps, x0, **kwargs):
        self.eps = eps
        self.x0 = x0
        UserExpression.__init__(self, **kwargs)
    def eval(self, values, x):
        eps = self.eps
        values[0] = eps/pi/(np.linalg.norm(x-self.x0)**2 + eps**2)
        values[1] = 0
    def value_shape(self): return (2, )

# Obtain max, total and average displacements
def get_displacement_measures(u, V):
    u_magnitude = sqrt(dot(u, u))
    u_magnitude = project(u_magnitude, V)
    u_magnitude = u_magnitude.vector()
    disp_len = u_magnitude.size()
    disp_max = u_magnitude.max()   # maximal displacement
    disp_tot = u_magnitude.sum()   # total/sum displacement
    disp_avg = disp_tot / disp_len # average displacement
    return disp_max, disp_tot, disp_avg

# returns displacements and vm stress for side-compression of the square
def square_FEA(mesh, verbose=False):
    if not verbose:
        set_log_active(False)  # dolfin verbose to false
    else:
        set_log_active(True)   # dolfin verbose to true

    # Define function space for system of PDEs
    degree = 2
    lambda_ = 1
    mu = 1
    V = VectorFunctionSpace(mesh, 'P', degree)

    # Define boundary conditions
    bc1 = DirichletBC(V, Constant((0, 0)), clamped_middle)
    bcs = [bc1]  # combined boundary conditions
    delta = Delta(eps=1E-4, x0=np.array([7.5, 4]), degree=5)  # point load at x0

    # Define variational problem
    u = TrialFunction(V)
    d = u.geometric_dimension()   # space dimension
    v = TestFunction(V)
    f = Constant((0, 0))
    T = Constant((0, 0))
    a = inner(sigma(u, lambda_, d, mu), epsilon(v)) * dx
    L = inner(Constant(-150) * delta, v) * dx

    # Compute solution
    u = Function(V)
    solve(a == L, u, bcs)

    # Displacement measures
    V1 = FunctionSpace(mesh, 'P', 1)
    disp_max, disp_tot, disp_avg = get_displacement_measures(u, V1)

    # max Von Mises Stress measure
    s = sigma(u, lambda_, d, mu) - (1./3) * tr(sigma(u, lambda_, d, mu)) * Identity(d)  # deviatoric stress
    von_mises = sqrt(3./2 * inner(s, s))
    von_mises = project(von_mises, V1)
    max_von_mises = von_mises.vector().max()  # getting maximal von Mises stress

    return disp_max, disp_tot, disp_avg, max_von_mises


In [3]:
# Single iteration from Gmsh meshing to FEniCS in functions
import meshio
import gmsh
from ufl import nabla_div

cwd = Path.cwd()
mesh_dir = cwd.joinpath("MOEA_meshes", "smallSquare3x3")  # adjust the dir here
inf_file = str(mesh_dir.stem) + '-infill.msh'
mesh_inp = mesh_dir / inf_file

# Extracting geometry from the infill mesh
conn, coords, num_el = extract_mesh_data(mesh_inp)

# Create the mesh infill based on the infill vector
vec = [0,0,1,1,0,0,0,0,1,1,0,0,0,0,1,1,0,0]
meshfile = mesh_dir / 'infill_gen.msh'
generate_infill_file(meshfile, vec, conn, coords, run_GUI=False)

# Loading the created infill mesh to FEniCS
mesh, _, _, _ = extract_mesh_data_fenics(meshfile)

# perform FEA and obtain displacements and von mises stress
d_max, d_tot, d_avg, max_vm = square_FEA(mesh, verbose=False)

print(d_max)
print(max_vm)

2.3813627312677026
0.3697625990138598


In [ ]:
# implementing pymoo multi-objective optimization with FEniCS FEA objectives

import numpy as np
import matplotlib.pyplot as plt
import pathlib
from pathlib import Path

from pymoo.core.problem import ElementwiseProblem
from pymoo.algorithms.moo.nsga2 import NSGA2

from pymoo.operators.sampling.rnd import BinaryRandomSampling
from pymoo.operators.crossover.hux import HalfUniformCrossover
from pymoo.operators.crossover.ux import UniformCrossover
from pymoo.operators.mutation.bitflip import BitflipMutation

from pymoo.termination import get_termination
from pymoo.optimize import minimize


# defining the multiobjective problem for square FEA problem: Weight/vm_stress obj
class MyProblem(ElementwiseProblem):

    def __init__(self, conn, coords, num_el, meshpath):
        super().__init__(n_var=num_el,
                         n_obj=2,
                         n_ieq_constr=1,
                         xl=np.zeros(num_el),
                         xu=np.ones(num_el)
                         )
        self.conn = conn
        self.coords = coords
        self.num_el = num_el
        self.meshpath = meshpath  # temp mesh for evaluation

    def _evaluate(self, x, out, *args, **kwargs):

        # generate the infill file
        generate_infill_file(self.meshpath, x, self.conn, self.coords)

        # Loading the created infill mesh to FEniCS
        mesh, _, _, _ = extract_mesh_data_fenics(meshfile)

        # perform FEA and obtain displacements and von mises stress
        d_max, d_tot, d_avg, max_vm = square_FEA(mesh)

        # objective values: we want to minimize *weight* and *maximal stress*
        f1 = np.sum(x)  # objective 1 - "weight"
        f2 = max_vm     # objective 2 - "maxStress"

        # constraints
        g1 = np.sum(x) - 20   # dummy constraint: can be adjusted to limit the infill rate

        out["F"] = [f1, f2]  # dictionary key for objectives
        out["G"] = [g1]      # dictionary key for constraints


# Filepath management
cwd = Path.cwd()
mesh_dir = cwd.joinpath("MOEA_meshes", "smallSquare3x3")  # adjust the dir here
inf_file = str(mesh_dir.stem) + '-infill.msh'
mesh_inp = mesh_dir / inf_file  # mesh input file

# Extracting geometry from the infill mesh -> num_el is the design dimension
conn, coords, num_el = extract_mesh_data(mesh_inp)   # extracting the geometry

# init the MOP
meshfile = mesh_dir / 'infill_gen.msh'
problem = MyProblem(conn, coords, num_el, meshfile)


# initialize the algorithm object
algorithm = NSGA2(
    pop_size=100,
    n_offsprings=40,
    sampling=BinaryRandomSampling(),
    crossover=HalfUniformCrossover(prob_hux=0.5),
    #crossover=UniformCrossover(),
    mutation=BitflipMutation(prob=0.5, prob_var=0.3),
    eliminate_duplicates=True
)


# define the termination criterion
termination = get_termination("n_gen", 200)


# solve the MOP
res = minimize(problem,
               algorithm,
               termination,
               seed=1,
               save_history=True,
               verbose=True)

# n_gen  - generation counter,
# n_eval - number of evaluations
# cv_min - minimum constraint violation
# cv_avg - average constraint violation in the current population.
# n_nds  - number of non-dominated solutions
# eps/ind- running performance metrics

X = res.X  # solutions in the decision space
F = res.F  # solutions in the objective space
hist = res.history  # getting the info for each iteration

#print(X)
#print(F)

# save solutions
out_dir = mesh_dir / "infill_gen"
out_dir.mkdir(exist_ok=True)
for i, x in enumerate(X):
    ifile = out_dir / f"sol{i:02d}.msh"
    generate_infill_file(ifile, x, conn, coords)
    print(f"File {ifile} saved.")###

# writing to a file
with open(out_dir / "sq_solutions.txt", "w+") as file:
    file.writelines(f"{str(list(sol))} \t w: {F[i][0]:10.1f} \t vm: {F[i][1]:10.06f} \n" for i, sol in enumerate(1*X))

# plotting the decision space
xl, xu = problem.bounds()
plt.figure(figsize=(7, 5))
plt.imshow(X)
plt.title("Design Space")
plt.show()


# plotting the objective space
plt.figure(figsize=(7, 5))
plt.scatter(F[:, 0], F[:, 1], s=30, facecolors='none', edgecolors='blue')
plt.title("Objective Space")
plt.show()

n_gen  |  n_eval  | n_nds  |     cv_min    |     cv_avg    |      eps      |   indicator  
     1 |      100 |      6 |  0.000000E+00 |  0.000000E+00 |             - |             -
     2 |      140 |      6 |  0.000000E+00 |  0.000000E+00 |  0.3277559889 |         nadir
     3 |      180 |      6 |  0.000000E+00 |  0.000000E+00 |  0.1111111111 |         ideal
     4 |      220 |      6 |  0.000000E+00 |  0.000000E+00 |  0.0009458589 |             f
     5 |      260 |      6 |  0.000000E+00 |  0.000000E+00 |  0.1000000000 |         ideal
     6 |      300 |      6 |  0.000000E+00 |  0.000000E+00 |  0.0035106224 |             f
     7 |      340 |      6 |  0.000000E+00 |  0.000000E+00 |  0.0136639920 |             f
     8 |      380 |      8 |  0.000000E+00 |  0.000000E+00 |  0.0259475876 |             f
     9 |      420 |      8 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
    10 |      460 |      9 |  0.000000E+00 |  0.000000E+00 |  0.0909090909 |         ideal

---
## Different Sized infill meshes for **Square Outline**
~ 100, 1000, 5000, 10000 elements

In [40]:
# SQUARE-OUTLINE + SQUARE-INFILL mesh setup which will work with FEniCS/Dolphin
import gmsh
import pathlib
from pathlib import Path

size = 8  # physical dimension of the side
P = 1     # number of perimeters
width, height = size, size       # setting up the square
num_el_x, num_el_y = size, size  # adjust the num of elements
num_el_x_inf, num_el_y_inf = 3, 3  # adjust num of elems for infill
dx, dy = width / num_el_x, height / num_el_y


# --- Create the Outline Mesh --------------------------------

# filepath management
path = Path.cwd()
parent_dir = path.joinpath("MOEA_meshes")
parent_dir.mkdir(exist_ok=True)          # create parent dir if it does not exist
meshname = "smallSquare3x3"              # gmsh model name
mesh_dir = parent_dir.joinpath(meshname) # setting up dir for meshes
mesh_dir.mkdir(exist_ok=True)            # create mesh dir if it does not exist



# initialize Gmsh
if not gmsh.is_initialized():
    gmsh.initialize()

gmsh.model.add(meshname)

# preparing parameters
lc = 2 * dx     # mesh size

# defining a namespace
gmg = gmsh.model.geo  # gmg ~ gmsh.model.geo

# defining the points
gmg.addPoint(0, 0, 0, lc, 1)
gmg.addPoint(0, size, 0, lc, 2)
gmg.addPoint(size, size, 0, lc, 3)
gmg.addPoint(size, 0, 0, lc, 4)
gmg.addPoint(0 + P * dx, 0 + P * dy, 0, lc, 5)
gmg.addPoint(0 + P * dx, size - P * dy, 0, lc, 6)
gmg.addPoint(size - P * dx, size - P * dy, 0, lc, 7)
gmg.addPoint(size - P * dx, 0 + P * dy, 0, lc, 8)



# adding lines
gmg.addLine(1, 2, 1)
gmg.addLine(2, 3, 2)
gmg.addLine(3, 4, 3)
gmg.addLine(4, 1, 4)

gmg.addLine(5, 6, 5)
gmg.addLine(6, 7, 6)
gmg.addLine(7, 8, 7)
gmg.addLine(8, 5, 8)

# defining a new surface:
gmg.addCurveLoop([1, 2, 3, 4], 20)
gmg.addCurveLoop([5, 6, 7, 8], 21)


gmg.addPlaneSurface([20, 21], 1)  # perimeter surface
#gmg.addPlaneSurface([24], 5)  # inner/infill surface   ### INFILL

# synchronize the model
gmg.synchronize()

# adding physical groups
# gmsh.model.addPhysicalGroup(1, [1, 2, 3, 4, 5, 6, 7, 8], name="outerLoop") # curves physical group
# gmsh.model.addPhysicalGroup(1, [9, 10, 11, 12], name="innerLoop") # curves physical group
# gmsh.model.addPhysicalGroup(2, [1, 2, 3, 4], name="PerimeterSurface")   # surfaces physical group
gmg.synchronize()

# global meshing options
gmsh.option.setNumber("Mesh.Algorithm", 6)  # Frontal-Delaunay for 2D meshes

# generate the mesh
gmg.synchronize()
gmsh.model.mesh.generate(2)

# saving the mesh
gmsh.write(str(mesh_dir.joinpath(meshname + '.msh')))

# Launch the GUI
gmg.synchronize()
gmsh.fltk.run()

# finish the session
gmsh.finalize()


# --- Create the Infill Mesh ----------------------------------------

# initialize Gmsh
if not gmsh.is_initialized():
    gmsh.initialize()

meshname = meshname + "-infill"
gmsh.model.add(meshname)

# preparing parameters
Fn = 1       # finest of the mesh
lc = 2 * dx  # mesh size

# defining a namespace
gmg = gmsh.model.geo  # gmg ~ gmsh.model.geo

# defining the points
# gmg.addPoint(0, 0, 0, lc, 1)
# gmg.addPoint(0, size, 0, lc, 2)
# gmg.addPoint(size, size, 0, lc, 3)
# gmg.addPoint(size, 0, 0, lc, 4)
gmg.addPoint(0 + P * dx, 0 + P * dy, 0, lc, 5)
gmg.addPoint(0 + P * dx, size - P * dy, 0, lc, 6)
gmg.addPoint(size - P * dx, size - P * dy, 0, lc, 7)
gmg.addPoint(size - P * dx, 0 + P * dy, 0, lc, 8)



# adding lines
# gmg.addLine(1, 2, 1)
# gmg.addLine(2, 3, 2)
# gmg.addLine(3, 4, 3)
# gmg.addLine(4, 1, 4)

gmg.addLine(5, 6, 5)
gmg.addLine(6, 7, 6)
gmg.addLine(7, 8, 7)
gmg.addLine(8, 5, 8)

# defining a new surface:
gmg.addCurveLoop([5, 6, 7, 8], 22)
gmg.addPlaneSurface([22], 1)  # infill surface

# synchronize the model
gmg.synchronize()

gmg.mesh.setTransfiniteCurve(5, Fn * (num_el_y_inf + 1))
gmg.mesh.setTransfiniteCurve(6, Fn * (num_el_x_inf + 1))
gmg.mesh.setTransfiniteCurve(7, Fn * (num_el_y_inf + 1))
gmg.mesh.setTransfiniteCurve(8, Fn * (num_el_x_inf + 1))
gmg.synchronize()

# meshing the transfinite surface
gmg.mesh.setTransfiniteSurface(1, "Alternate", [5, 6, 7, 8])  # set up the corners of interpolation   ### INFILL

# synchronize the model
gmg.synchronize()


# adding physical groups
# gmsh.model.addPhysicalGroup(2, [1], name="InfillSurface")   # surfaces physical group
gmg.synchronize()

# global meshing options
gmsh.option.setNumber("Mesh.Algorithm", 6)  # Frontal-Delaunay for 2D meshes

# generate the mesh
gmg.synchronize()
gmsh.model.mesh.generate(2)

# saving the mesh
gmsh.write(str(mesh_dir.joinpath(meshname + '.msh')))

# Launch the GUI
gmg.synchronize()
gmsh.fltk.run()

# finish the session
gmsh.finalize()


Info    : Meshing 1D...
Info    : [  0%] Meshing curve 5 (Line)
Info    : [ 30%] Meshing curve 6 (Line)
Info    : [ 50%] Meshing curve 7 (Line)
Info    : [ 80%] Meshing curve 8 (Line)
Info    : Done meshing 1D (Wall 0.000183432s, CPU 0.000294s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Transfinite)
Info    : Done meshing 2D (Wall 3.163e-05s, CPU 3e-05s)
Info    : 16 nodes 34 elements
Info    : Writing '/Users/macbookpro/Desktop/Project2023/MOEA_meshes/smallSquare3x3/smallSquare3x3-infill.msh'...
Info    : Done writing '/Users/macbookpro/Desktop/Project2023/MOEA_meshes/smallSquare3x3/smallSquare3x3-infill.msh'
-------------------------------------------------------
Version       : 4.11.1
License       : GNU General Public License
Build OS      : MacOSX-sdk
Build date    : 20230510
Build host    : Mac-1683750338053.local
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack MathEx[

In [27]:
# pathlib tutorial
# filepath management
path = Path.cwd()
mesh_folder = path.joinpath("MOEA_meshes")
mesh_folder.mkdir(exist_ok=True)
print(mesh_dir)

/Users/macbookpro/Desktop/Project2023/MOEA_meshes/smallSquare3x3


In [30]:
fpath = mesh_dir / (meshname + '.msh')
msh = meshio.read(fpath)

In [31]:
fpath.parent

PosixPath('/Users/macbookpro/Desktop/Project2023/MOEA_meshes/smallSquare3x3')

In [46]:
cwd = Path.cwd()
mesh_dir  = cwd.joinpath("MOEA_meshes", "smallSquare3x3")

meshfile = mesh_dir / 'sq_infill_gen.msh'

meshfile.name[0]

's'

In [47]:
meshfile.stem

'sq_infill_gen'